In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

# Engine Setup 

Within this chapter the simplest form of a chess engine wil be developed, one that makes random moves. 
It will be possible to play against the engine or let two engines play against each others. 
As the goal is to incrementally enhance the chess engine, a base class for the Engine and a generic play function is implemented.

## Common Engine Interface

In general, all engines will need to represent a move along with a score. Therefore a simple data class `ScoredMove` is introduced. By using the `dataclass` decorater with `order=True` python comparison methods, 
such as `__lt__()`, will be auto-generated.
This allows to compare two scored moves or sort a list of scored moves.
The comparison is only based on the score as the move field has `compare=False`. 

In [ ]:
from dataclasses import dataclass, field
import chess


@dataclass(order=True)
class ScoredMove:
    """Class for representing a move along with a score."""
    score: int
    move: chess.Move = field(compare=False)

To simplify writing generic code for playing a chess game with different engines,
all engines will implement a common interface. 
The interface `Engine`,
which is highly inspired by `chess.engine`,
has two abstract functions: `play` and `analyse`.

The `play` function takes the current board as a parameter 
and returns a `chess.engine.PlayResult` object, 
which contains the next move 
and information if the engine offered a draw or resigned. 

The `analyse` function takes the board as a parameter as well 
and returns a list of `ScoredMove` objects.

In [ ]:
from abc import ABC, abstractmethod
import chess.engine


class Engine(ABC):
    """Common interface for all chess engines"""
    @abstractmethod
    def play(self, board: chess.Board) -> chess.engine.PlayResult:
        pass

    @abstractmethod
    def analyse(self, board: chess.Board) -> list[ScoredMove]:
        pass

## Random Engine

The `RandomEngine` inherits from `Engine` and implements both abstract methods. 

In the `play` function the engine radomly selects one of the legal moves provided by the `board` object and return it as a `chess.engine.PlayResult`.

The `analyse` function returns a list of all possible legal moves with a score of zero for each as the random engine does not evaluate the moves.

In [ ]:
import random
import chess.engine


class RandomEngine(Engine):
    """Basic chess engine playing a random legal move"""
    def play(self, board: chess.Board) -> chess.engine.PlayResult:
        move = random.choice(list(board.legal_moves))
        return chess.engine.PlayResult(move=move, ponder=None)

    def analyse(self, board: chess.Board) -> list[ScoredMove]:
        return [ScoredMove(0, move) for move in board.legal_moves]

To let two engines play against each other a function is created. 
The function `playGameSimple` shows a simple implementation, which takes a chess board and two engines as a parameter.
Until the game is over, the engines take turns making their moves.

In [ ]:
def playGameSimple(
    board: chess.Board, engine1: Engine, engine2: Engine
) -> chess.Board:
    """Function to play two Engines against each other"""
    engines = [engine1, engine2]

    while not board.is_game_over():
        move = engines[0].play(board).move
        board.push(move)
        engines[0], engines[1] = engines[1], engines[0]

    return board

A more enhanced version of this simple play function is `playGame`. 
Optionally, it can display the chess board after every move when passing `displayBoard = True` to the function. 
The second optional feature, enabled by passing `logMoves = True`, logs every move to a file with the current timestamp as the filename to ensure uniqueness. 

In [ ]:
import IPython.display
import time
from datetime import datetime


def displayChessBoard(board: chess.Board, display: bool = True):
    if display:
        IPython.display.clear_output(wait=True)
        IPython.display.display(board)
        time.sleep(0.5)


def logMove(
    board: chess.Board, move: chess.Move, fileName: str, log: bool = True
):
    if log:
        with open(fileName, "a") as f:
            if board.turn is chess.WHITE:
                f.write(board.lan(move) + "\t")
            else:
                f.write(board.lan(move) + "\n")


def playGame(
    board: chess.Board,
    engine1: Engine,
    engine2: Engine,
    displayBoard: bool = False,
    logMoves: bool = False
) -> chess.Board():
    """Function to play two Engines against each other
    
    Keyword arguments:
    displayBoard -- toggle to enable displaying the board after each half turn (default false)
    logMoves -- toggle to create a log of moves (default false)
    """
    engines = [engine1, engine2]

    logFileName = f"../log/{datetime.isoformat(datetime.now())}.txt"

    displayChessBoard(board, displayBoard)

    while not board.is_game_over():
        move = engines[0].play(board).move

        logMove(board, move, logFileName, logMoves)

        board.push(move)
        engines[0], engines[1] = engines[1], engines[0]

        displayChessBoard(board, displayBoard)
    return board

Next, 
the function is called 
with two `RandomEngine` objects 
and then prints the outcome of the board 
and the end position. 
When passsing `displayBoard=True` 
one can see that both engines play without any strategy.

The random number generator (RNG) depends on an inital seed. 
In the code the seed is explicitly set to create reproducible result.
For real world applications, the seed should be set by a high entropy source.

In [ ]:
random.seed(42)
board = chess.Board()
playGame(board, RandomEngine(), RandomEngine())
IPython.display.display(board)
print(board.outcome())

## Human Player 

Next, it should be possible to play against the random engine with human input. 
In ordner to be compatible with the previously defined `playGame` method, player input is realized as an engine too. 

In the `HumanEngine` class only the `play` method is defined, which takes the input in uci notation from the player. 
The `analyze` method reuses the implementation of the `RandomEngine` and therefore just returns every possible move with a score of zero.

In [ ]:
class HumanEngine(Engine):
    """Chess engine using the abilities of a human player"""
    """List of keywords to abort input"""
    ABORT_KEYWORDS = ["EXIT", "CANCEL"]

    def play(self, board: chess.Board) -> chess.engine.PlayResult:
        """Play method taking input of human players"""
        whitesTurn = board.turn is chess.WHITE
        legalMoves = board.legal_moves
        while True:
            enteredMove = input(
                f"Enter move for {'white' if whitesTurn else 'black'} in UCI notation: "
            )
            if enteredMove.upper() in HumanEngine.ABORT_KEYWORDS:
                raise Exception("User aborted")

            try:
                move = chess.Move.from_uci(enteredMove)
                if move in legalMoves:
                    return ScoredMove(0, move)
                print("Illegal input. Please try again.")
            except ValueError:
                print(f"Illegal notation. Valid moves are {list(legalMoves)}")

    analyse = RandomEngine.analyse

This engine can be used as shown in the following commented block.

In [ ]:
# random.seed(42)
# board = chess.Board()
# playGame(board, HumanEngine(), RandomEngine(), displayBoard=True)
# print(board.outcome())